### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np

In [2]:
bc = datasets.load_breast_cancer() #使用乳癌資料集
bc

{'data': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0

In [3]:
x_train, x_test, y_train, y_test = train_test_split(bc.data, bc.target, test_size=0.25, random_state=42)

# 建立模型
clf = GradientBoostingClassifier(random_state=7)

clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
auc = metrics.roc_auc_score(y_test, y_pred)
print(y_pred)
print("原本的auc: %f " %(auc))

      
threshold = 0.5 
y_pred_binarized = np.where(y_pred>threshold, 1, 0) # 使用 np.where 函數, 將 y_pred > 0.5 的值變為 1，小於 0.5 的為 0
f1 = metrics.f1_score(y_test, y_pred_binarized) # 使用 F1-Score 評估
precision = metrics.precision_score(y_test, y_pred_binarized) # 使用 Precision 評估
recall  = metrics.recall_score(y_test, y_pred_binarized) # 使用 recall 評估
print("F1-Score: ", f1) 
print("Precision: ", precision)
print("Recall: ", recall)

[1 0 0 1 1 0 0 0 0 1 1 0 1 0 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0
 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 1 0 0 1 1 1 0 0 1 1 0 0 1 0
 1 1 1 1 1 1 0 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 1 0 0 1 0 0 1 1 1 0 0 1 0
 1 1 0 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 0 0 0 1 1 1 0 0 1 1 0 1 0 1]
原本的auc: 0.955368 
F1-Score:  0.9662921348314607
Precision:  0.9662921348314607
Recall:  0.9662921348314607


In [4]:
n_estimators = [100, 200, 300, 400, 500]
max_depth = [1, 3, 5, 7, 9]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)
# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)
# 預設會跑 5-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 125 out of 125 | elapsed:   51.8s finished


In [5]:
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -0.028208 using {'max_depth': 1, 'n_estimators': 100}


In [6]:
grid_result.best_params_

{'max_depth': 1, 'n_estimators': 100}

In [7]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingRegressor(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)
y_pred

array([ 0.91098009,  0.02353915,  0.02770359,  0.95868704,  1.04307973,
       -0.13729944,  0.01330391,  0.15706536,  0.41581634,  0.96962136,
        0.87584206,  0.04386284,  0.81768372,  0.25976474,  1.05872226,
       -0.01162558,  0.89684126,  1.02487174,  1.03836547,  0.06152907,
        0.82711869,  0.96548842, -0.08792833,  0.9496341 ,  0.90171111,
        0.83352787,  1.02436866,  0.90014066,  0.98151528, -0.05273149,
        0.93501179,  1.05439234,  0.96080822,  0.89828687,  1.00884487,
        0.99461109,  0.47513726,  1.04908304,  0.01595656,  0.79878771,
        1.01240502, -0.03764417,  0.97479019,  1.00017781,  0.84955473,
        0.90444746,  0.94141817,  0.90036329,  0.88912714,  0.99811572,
       -0.05777753, -0.1067726 ,  0.78653947,  0.82512953,  1.0801883 ,
        0.96725825,  1.03486347, -0.01943618,  0.35167273,  1.05439234,
        0.9993139 , -0.00974816, -0.00705287,  0.86965858,  0.95482793,
        0.85474571, -0.0775157 , -0.08856461,  0.95125463,  0.90

In [8]:
print("最新的auc: %f " %(auc))

最新的auc: 0.955368 


In [9]:
threshold = 0.5 
y_pred_binarized = np.where(y_pred>threshold, 1, 0) # 使用 np.where 函數, 將 y_pred > 0.5 的值變為 1，小於 0.5 的為 0
f1 = metrics.f1_score(y_test, y_pred_binarized) # 使用 F1-Score 評估
precision = metrics.precision_score(y_test, y_pred_binarized) # 使用 Precision 評估
recall  = metrics.recall_score(y_test, y_pred_binarized) # 使用 recall 評估
print("F1-Score: ", f1) 
print("Precision: ", precision)
print("Recall: ", recall)

F1-Score:  0.9662921348314607
Precision:  0.9662921348314607
Recall:  0.9662921348314607
